# Python Bindings Playground

## Set Up the Environment

In [ ]:
import sys
import subprocess
from pathlib import Path

assert sys.prefix != sys.base_prefix, "You need to install Jupyter inside a virtual environment and run the notebook from there"

In [ ]:
!maturin develop --release

In [ ]:
!pip install --quiet scipy matplotlib

In [ ]:
status = subprocess.run(["git", "rev-parse", "--show-toplevel"], stdout=subprocess.PIPE, check=True)

RUNE_PROJECT_ROOT = Path(status.stdout.decode("utf-8").strip())
EXAMPLE_DIR = RUNE_PROJECT_ROOT / "examples"

## Fast Fourier Transform

In [ ]:
from rune_py import Fft
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
import math

In [ ]:
no = EXAMPLE_DIR / "microspeech" / "data" / "no_b66f4f93_nohash_8.wav"
samplerate, data = wavfile.read(no)

fft = Fft()

fft.sample_rate = samplerate
fft.smoothing_bits = 10
fft.even_smoothing = 0.025
fft.odd_smoothing = 0.06
fft.min_signal_remaining = 0.05
fft.strength = 0.95
fft.offset = 80.0
fft.gain_bits = 21

spectrum = fft(data).reshape(49, 40)

fig, axis = plt.subplots() 
heatmap = axis.pcolor(spectrum)
plt.colorbar(heatmap)
plt.show()

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, FloatSlider, FloatLogSlider

In [ ]:
def render(smoothing_bits, even_smoothing, odd_smoothing):
    fft = Fft()

    fft.sample_rate = samplerate
    fft.smoothing_bits = smoothing_bits
    fft.even_smoothing = even_smoothing
    fft.odd_smoothing = odd_smoothing
    fft.min_signal_remaining = 0.05
    fft.strength = 0.95
    fft.offset = 80.0
    fft.gain_bits = 21

    spectrum = fft(data).reshape(49, 40)

    plt.imshow(spectrum)
    plt.show()
    
interact(
    render, 
    smoothing_bits=IntSlider(min=0, max=16, value=10),
    even_smoothing=FloatLogSlider(min=-2, max=2, value=0),
    odd_smoothing=FloatLogSlider(min=-2, max=2, value=0),
);